In [2]:
import numpy as np
from sklearn.decomposition import PCA
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ModuleNotFoundError: No module named 'ibm_quantum_widgets'

In [ ]:
# Example matrix
a = np.array([[126, 78], [128, 80], [128, 82], [130, 82], [130, 84],  [132, 86]], dtype=np.int32)
display(a)

array([[126,  78],
       [128,  80],
       [128,  82],
       [130,  82],
       [130,  84],
       [132,  86]], dtype=int32)

In [ ]:
#Let's create a function to do this for us:
class new_PCA:
    def __init__(self, matrix):
        self.matrix = matrix
        self.centered = self.center_matrix(matrix)
        self.covariance = np.cov(self.centered, rowvar=False)
        self.eigen = np.linalg.eig(self.covariance)
        self.eigenvalues, self.eigenvectors = self.organizer(self.eigen)
        self.projection = self.PCA(self.centered, self.eigenvectors)
    
    def center_matrix(self, matrix):
        return matrix - matrix.mean(axis=0)

    
    def organizer(self, eigen):
        index_list = np.flip(np.argsort(eigen[0]))
        newvectors = np.empty((0,2))
        eigen_vectors = eigen[1].transpose()
        for i in index_list:
            newvectors = np.append(newvectors, [eigen_vectors[i]], axis=0)
        return np.flip(np.sort(eigen[0])), newvectors.transpose()
    
    
    def PCA(self, centered, eigenvectors):
        return centered.dot(eigenvectors)

In [ ]:
pca_matrix = new_PCA(a)
pca_matrix.projection
print("eigenvectors:\n", pca_matrix.eigenvectors)
print("eigenvalues:\n", pca_matrix.eigenvalues)
print("Principal transformation:\n", pca_matrix.projection)

eigenvectors:
 [[-0.58906316 -0.80808699]
 [-0.80808699  0.58906316]]
eigenvalues:
 [12.08217647  0.31782353]
Principal transformation:
 [[ 4.99953747  0.06800833]
 [ 2.20523715 -0.37003933]
 [ 0.58906316  0.80808699]
 [-0.58906316 -0.80808699]
 [-2.20523715  0.37003933]
 [-4.99953747 -0.06800833]]


In [ ]:
# PCA Ready module. We'll use as benchmark.
oldPCA = PCA()
oldPCA.fit(a)
print("eigenvectors:\n", oldPCA.components_)
print("eigenvalues:\n", oldPCA.explained_variance_)
print("Principal transformation:\n", oldPCA.transform(a))

eigenvectors:
 [[-0.58906316 -0.80808699]
 [-0.80808699  0.58906316]]
eigenvalues:
 [12.08217647  0.31782353]
Principal transformation:
 [[ 4.99953747  0.06800833]
 [ 2.20523715 -0.37003933]
 [ 0.58906316  0.80808699]
 [-0.58906316 -0.80808699]
 [-2.20523715  0.37003933]
 [-4.99953747 -0.06800833]]


In [ ]:
# Did it work?? (Values may not be exactly the same.
np.allclose(oldPCA.transform(a), pca_matrix.projection)

True

In [ ]:
b = np.array([[1., 2.], [3., 4.], [5., 6.]])
display(b)

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

In [ ]:
b_newpca = new_PCA(b)
display(b_newpca.projection)
b_pca = PCA()
display(b_pca.fit_transform(b))

array([[-2.82842712,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.82842712,  0.        ]])

array([[-2.82842712,  0.        ],
       [ 0.        ,  0.        ],
       [ 2.82842712,  0.        ]])

In [3]:
c = np.array([[1.5, 0.5], [0.5, 1.5]])
display(c)

print("MYPCA")
c_newpca = new_PCA(c)
display(c_newpca.projection)
display(c_newpca.eigenvalues)
display(c_newpca.eigenvectors)

print("TheirPCA")
c_pca = PCA()
display(c_pca.fit_transform(c))
display(c_pca.explained_variance_)
display(oldPCA.components_)

array([[1.5, 0.5],
       [0.5, 1.5]])

MYPCA


NameError: name 'new_PCA' is not defined

# Time to implement quantum PCA:

In [21]:
# We need to generate our states
# We create the following function that will return an NP array with the set state vector.

def state_prep(matrix):
    flat = np.array([matrix.flatten()])
    sumsq = np.sqrt(np.sum(matrix**2))
    return flat.T/sumsq

In [33]:
state_prep(c)

array([[0.67082039],
       [0.2236068 ],
       [0.2236068 ],
       [0.67082039]])